In [39]:
import pandas as pd
import os
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import json



from sklearn.datasets import fetch_20newsgroups

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

import re

from sklearn.linear_model import SGDClassifier

from sklearn import metrics

from sklearn.ensemble import VotingClassifier

In [2]:
def regex_condi(string):
    if ".com" in string:
        return False
    if ".edu" in string:
        return False
    if "@" in string:
        return False
    if 'Host'in string:
        return False
    if ".gov" in string:
        return False

    return True

def clean_text(string):
    string = " ".join([i for i in string.split() if regex_condi(i)])
    string = re.sub(r"Re:","", string)
    string = re.sub(r"Reply-To:","", string)
    toreturn = string.strip()
    return toreturn

In [3]:
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

newsgroups_train_X,newsgroups_train_y = newsgroups_train.data, newsgroups_train.target
newsgroups_val_X, newsgroups_test_X, newsgroups_val_y, newsgroups_test_y = train_test_split(newsgroups_test.data, newsgroups_test.target, test_size=0.5, random_state=42)


# Clean train 
newsgroups_train_X = [clean_text(i) for i in newsgroups_train_X]
newsgroups_val_X = [clean_text(i) for i in newsgroups_val_X]
newsgroups_test_X =  [clean_text(i) for i in newsgroups_test_X]

False

In [134]:
def BoG_model(X,y, clf = 'Logistic', vocab = None, combining = False):
    """
    This function take in training data as X, y. 
    Then the data will be processed with BagOfWord, then feed into the Logistic / SGD Classifier.

    Return:
        pipe: The fitted Logistic Classifier.
    """
    if vocab is None:
        if clf == 'Logistic':
            pipe = Pipeline([
               ('BagOfWord', CountVectorizer()),
               ('clasiffier', LogisticRegression())
               ])
        if clf == 'SVM':
            pipe = Pipeline([
               ('BagOfWord', CountVectorizer()),
               ('clasiffier', SGDClassifier())
               ])
    else:
        if combining == True:
            count_vect = CountVectorizer().fit(X)   
            vocab_lst = np.unique(list(count_vect.vocabulary_) + list(vocab))
        if combining == False:
            vocab_lst = vocab
            
        if clf == 'Logistic':
            pipe = Pipeline([
               ('BagOfWord', CountVectorizer(vocabulary =vocab_lst)),
               ('clasiffier', LogisticRegression())
               ])
        if clf == 'SVM':
            pipe = Pipeline([
               ('BagOfWord', CountVectorizer(vocabulary =vocab_lst)),
               ('clasiffier', SGDClassifier())
               ])
    pipe.fit(X, y)
    return pipe

def Tfidf_model(X,y, clf = 'Logistic', vocab = None, combining = False):

    """
    This function take in training data as X, y. 
    Then the data will be processed with BagOfWord and Tf-Idf, then feed into the Logistic Classifier.

    Return:
        pipe: The fitted Logistic Classifier.
    """
    if vocab is None:
        if clf == 'Logistic':
            pipe = Pipeline([
                ('BagOfWord', CountVectorizer()),
                ('TfIdf',TfidfTransformer()),
                ('clasiffier', LogisticRegression())
                ])
        if clf == 'SVM':
            pipe = Pipeline([
                ('BagOfWord', CountVectorizer()),
                ('TfIdf',TfidfTransformer()),
                ('clasiffier', SGDClassifier())
                ])
    else:
        if combining == True:
            count_vect = CountVectorizer().fit(X)   
            vocab_lst = np.unique(list(count_vect.vocabulary_) + list(vocab))
        if combining == False:
            
            vocab_lst = vocab
            
        if clf == 'Logistic':
            pipe = Pipeline([
                ('BagOfWord', CountVectorizer(vocabulary =vocab_lst)),
                ('TfIdf',TfidfTransformer()),
                ('clasiffier', LogisticRegression())
                ])
        if clf == 'SVM':
            pipe = Pipeline([
                ('BagOfWord', CountVectorizer(vocabulary =vocab_lst)),
                ('TfIdf',TfidfTransformer()),
                ('clasiffier', SGDClassifier())
                ])
    pipe.fit(X, y)
    return pipe


In [6]:
Log_Count_clf = BoG_model(newsgroups_train_X, newsgroups_train_y)
predicted = Log_Count_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

C:\Users\Yang Li\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7774827403080191

In [7]:
print(metrics.classification_report(newsgroups_val_y, predicted,target_names=newsgroups_train.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.67      0.68      0.68       161
           comp.graphics       0.67      0.75      0.71       189
 comp.os.ms-windows.misc       0.73      0.66      0.69       197
comp.sys.ibm.pc.hardware       0.71      0.69      0.70       196
   comp.sys.mac.hardware       0.75      0.79      0.77       196
          comp.windows.x       0.77      0.71      0.74       202
            misc.forsale       0.81      0.85      0.83       206
               rec.autos       0.80      0.85      0.83       189
         rec.motorcycles       0.90      0.90      0.90       194
      rec.sport.baseball       0.87      0.86      0.87       192
        rec.sport.hockey       0.91      0.89      0.90       203
               sci.crypt       0.91      0.86      0.88       194
         sci.electronics       0.68      0.72      0.70       197
                 sci.med       0.73      0.73      0.73       196
         

In [8]:
Log_Count_clf['BagOfWord'].vocabulary_.get(u'algorithm')

17952

In [9]:
Log_TFIDF_clf = Tfidf_model(newsgroups_train_X, newsgroups_train_y)
predicted = Log_TFIDF_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

0.8266064790228359

In [10]:
print(metrics.classification_report(newsgroups_val_y, predicted,target_names=newsgroups_train.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.71      0.69      0.70       161
           comp.graphics       0.73      0.83      0.78       189
 comp.os.ms-windows.misc       0.77      0.73      0.75       197
comp.sys.ibm.pc.hardware       0.73      0.76      0.74       196
   comp.sys.mac.hardware       0.84      0.81      0.83       196
          comp.windows.x       0.86      0.77      0.81       202
            misc.forsale       0.78      0.91      0.84       206
               rec.autos       0.88      0.88      0.88       189
         rec.motorcycles       0.96      0.93      0.95       194
      rec.sport.baseball       0.90      0.95      0.93       192
        rec.sport.hockey       0.98      0.93      0.95       203
               sci.crypt       0.94      0.90      0.92       194
         sci.electronics       0.75      0.82      0.78       197
                 sci.med       0.88      0.81      0.85       196
         

In [11]:
SVM_Count_clf = BoG_model(newsgroups_train_X, newsgroups_train_y, clf = 'SVM')
predicted = SVM_Count_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

0.7461497610196495

In [12]:
print(metrics.classification_report(newsgroups_val_y, predicted,target_names=newsgroups_train.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.66      0.75      0.70       161
           comp.graphics       0.58      0.77      0.66       189
 comp.os.ms-windows.misc       0.63      0.59      0.61       197
comp.sys.ibm.pc.hardware       0.45      0.73      0.55       196
   comp.sys.mac.hardware       0.78      0.78      0.78       196
          comp.windows.x       0.84      0.72      0.77       202
            misc.forsale       0.87      0.73      0.79       206
               rec.autos       0.88      0.77      0.82       189
         rec.motorcycles       0.86      0.90      0.88       194
      rec.sport.baseball       0.92      0.82      0.87       192
        rec.sport.hockey       0.93      0.88      0.90       203
               sci.crypt       0.86      0.86      0.86       194
         sci.electronics       0.74      0.59      0.66       197
                 sci.med       0.84      0.72      0.78       196
         

In [13]:
SVM_TFIDF_clf = Tfidf_model(newsgroups_train_X, newsgroups_train_y, clf = 'SVM')
SVM_TFIDF_clf.fit(newsgroups_train_X, newsgroups_train_y)
predicted = SVM_TFIDF_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

0.8459904407859798

In [14]:
print(metrics.classification_report(newsgroups_val_y, predicted,target_names=newsgroups_train.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.75      0.76      0.75       161
           comp.graphics       0.77      0.84      0.80       189
 comp.os.ms-windows.misc       0.77      0.73      0.75       197
comp.sys.ibm.pc.hardware       0.74      0.77      0.75       196
   comp.sys.mac.hardware       0.84      0.85      0.84       196
          comp.windows.x       0.90      0.77      0.83       202
            misc.forsale       0.87      0.89      0.88       206
               rec.autos       0.89      0.88      0.89       189
         rec.motorcycles       0.95      0.94      0.95       194
      rec.sport.baseball       0.92      0.95      0.94       192
        rec.sport.hockey       0.97      0.99      0.98       203
               sci.crypt       0.92      0.93      0.92       194
         sci.electronics       0.81      0.77      0.79       197
                 sci.med       0.89      0.88      0.88       196
         

In [223]:
news_df

,data,filenames,target_names,target,DESCR
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,C:\Users\Yang Li\scikit_learn_data\20news_home...,alt.atheism,7,.
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,C:\Users\Yang Li\scikit_learn_data\20news_home...,comp.graphics,4,.
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,C:\Users\Yang Li\scikit_learn_data\20news_home...,comp.os.ms-windows.misc,4,
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,C:\Users\Yang Li\scikit_learn_data\20news_home...,comp.sys.ibm.pc.hardware,1,_
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,C:\Users\Yang Li\scikit_learn_data\20news_home...,comp.sys.mac.hardware,14,2
...,...,...,...,...,...
11309,From: jim.zisfein@factory.com (Jim Zisfein) \n...,C:\Users\Yang Li\scikit_learn_data\20news_home...,None,13,None
11310,From: ebodin@pearl.tufts.edu\nSubject: Screen ...,C:\Users\Yang Li\scikit_learn_data\20news_home...,None,4,None
11311,From: westes@netcom.com (Will Estes)\nSubject:...,C:\Users\Yang Li\scikit_learn_data\20news_home...,None,3,None
11312,From: steve@hcrlgw (Steven Collins)\nSubject: ...,C:\Users\Yang Li\scikit_learn_data\20news_home...,None,1,None


In [15]:
os.makedirs('../data/temp',exist_ok=True)

In [16]:
news_df = pd.DataFrame.from_dict(newsgroups_train,'index').T

In [17]:
with open('../data/temp/20news.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(news_df.data.apply(lambda x: clean_text(x)+ '.')))

In [18]:
df = pd.read_csv('../data/temp/AutoPhrase.txt',error_bad_lines=False, delimiter= '\t',names = ['p','phrase'])
select = df[df.p >= 0.5]
df

,p,phrase
0,0.988864,george bush
1,0.987358,red sox
2,0.986398,attorney general
3,0.985885,gulf war
4,0.985655,silicon graphics
...,...,...
12030,0.003707,not exactly
12031,0.003279,can't do
12032,0.003142,r +
12033,0.003109,probably not


In [19]:
count_vect = CountVectorizer(vocabulary = select.phrase,stop_words = 'english')
count_vect = count_vect.fit_transform(newsgroups_train_X)

In [20]:
auto_Log_Count_clf = BoG_model(newsgroups_train_X, newsgroups_train_y, vocab=select.phrase)
predicted = auto_Log_Count_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

C:\Users\Yang Li\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7732342007434945

In [21]:
print(metrics.classification_report(newsgroups_val_y, predicted,target_names=newsgroups_train.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.66      0.66      0.66       161
           comp.graphics       0.69      0.74      0.71       189
 comp.os.ms-windows.misc       0.72      0.67      0.69       197
comp.sys.ibm.pc.hardware       0.69      0.69      0.69       196
   comp.sys.mac.hardware       0.76      0.78      0.77       196
          comp.windows.x       0.80      0.71      0.75       202
            misc.forsale       0.79      0.84      0.81       206
               rec.autos       0.81      0.85      0.83       189
         rec.motorcycles       0.89      0.88      0.88       194
      rec.sport.baseball       0.89      0.88      0.88       192
        rec.sport.hockey       0.95      0.90      0.92       203
               sci.crypt       0.89      0.88      0.89       194
         sci.electronics       0.65      0.68      0.66       197
                 sci.med       0.77      0.75      0.76       196
         

In [22]:
auto_Log_Count_clf['BagOfWord'].vocabulary_.get(u'algorithm')

3304

In [23]:
auto_Log_TFIDF_clf = Tfidf_model(newsgroups_train_X, newsgroups_train_y, vocab=select.phrase)
predicted = auto_Log_TFIDF_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

0.8048327137546468

In [24]:
print(metrics.classification_report(newsgroups_val_y, predicted,target_names=newsgroups_train.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.68      0.68      0.68       161
           comp.graphics       0.70      0.79      0.74       189
 comp.os.ms-windows.misc       0.73      0.73      0.73       197
comp.sys.ibm.pc.hardware       0.69      0.72      0.70       196
   comp.sys.mac.hardware       0.81      0.79      0.80       196
          comp.windows.x       0.85      0.75      0.79       202
            misc.forsale       0.82      0.88      0.85       206
               rec.autos       0.87      0.84      0.85       189
         rec.motorcycles       0.93      0.92      0.93       194
      rec.sport.baseball       0.91      0.93      0.92       192
        rec.sport.hockey       0.95      0.94      0.95       203
               sci.crypt       0.94      0.89      0.91       194
         sci.electronics       0.69      0.75      0.72       197
                 sci.med       0.83      0.80      0.81       196
         

In [25]:
auto_SVM_Count_clf = BoG_model(newsgroups_train_X, newsgroups_train_y, vocab=select.phrase, clf = 'SVM')
predicted = auto_SVM_Count_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

0.7485395645246946

In [26]:
print(metrics.classification_report(newsgroups_val_y, predicted,target_names=newsgroups_train.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.66      0.66      0.66       161
           comp.graphics       0.67      0.76      0.71       189
 comp.os.ms-windows.misc       0.62      0.62      0.62       197
comp.sys.ibm.pc.hardware       0.60      0.58      0.59       196
   comp.sys.mac.hardware       0.74      0.77      0.76       196
          comp.windows.x       0.77      0.69      0.73       202
            misc.forsale       0.71      0.83      0.76       206
               rec.autos       0.75      0.82      0.78       189
         rec.motorcycles       0.88      0.84      0.86       194
      rec.sport.baseball       0.84      0.85      0.84       192
        rec.sport.hockey       0.91      0.93      0.92       203
               sci.crypt       0.91      0.82      0.86       194
         sci.electronics       0.71      0.66      0.68       197
                 sci.med       0.80      0.71      0.75       196
         

In [27]:
auto_SVM_TFIDF_clf = Tfidf_model(newsgroups_train_X, newsgroups_train_y, vocab=select.phrase, clf = 'SVM')
predicted = auto_SVM_TFIDF_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

0.8127987254381307

In [28]:
print(metrics.classification_report(newsgroups_val_y, predicted,target_names=newsgroups_train.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.66      0.71      0.68       161
           comp.graphics       0.72      0.78      0.75       189
 comp.os.ms-windows.misc       0.73      0.73      0.73       197
comp.sys.ibm.pc.hardware       0.75      0.71      0.73       196
   comp.sys.mac.hardware       0.78      0.81      0.79       196
          comp.windows.x       0.85      0.75      0.80       202
            misc.forsale       0.84      0.87      0.86       206
               rec.autos       0.86      0.84      0.85       189
         rec.motorcycles       0.92      0.92      0.92       194
      rec.sport.baseball       0.94      0.90      0.92       192
        rec.sport.hockey       0.93      0.98      0.95       203
               sci.crypt       0.90      0.91      0.91       194
         sci.electronics       0.74      0.74      0.74       197
                 sci.med       0.86      0.82      0.84       196
         

In [29]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(newsgroups_train_X)
all_vocab = np.unique(list(count_vect.vocabulary_) + list(select.phrase))

In [30]:
all_Log_Count_clf = BoG_model(newsgroups_train_X, newsgroups_train_y, vocab=all_vocab)
predicted = all_Log_Count_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

C:\Users\Yang Li\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7774827403080191

In [31]:
print(metrics.classification_report(newsgroups_val_y, predicted,target_names=newsgroups_train.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.64      0.70      0.67       161
           comp.graphics       0.68      0.76      0.72       189
 comp.os.ms-windows.misc       0.74      0.66      0.70       197
comp.sys.ibm.pc.hardware       0.70      0.71      0.71       196
   comp.sys.mac.hardware       0.75      0.79      0.77       196
          comp.windows.x       0.77      0.71      0.74       202
            misc.forsale       0.82      0.84      0.83       206
               rec.autos       0.79      0.85      0.82       189
         rec.motorcycles       0.90      0.90      0.90       194
      rec.sport.baseball       0.87      0.86      0.87       192
        rec.sport.hockey       0.92      0.89      0.90       203
               sci.crypt       0.92      0.86      0.89       194
         sci.electronics       0.70      0.71      0.70       197
                 sci.med       0.76      0.71      0.73       196
         

In [32]:
all_Log_TFIDF_clf = Tfidf_model(newsgroups_train_X, newsgroups_train_y, vocab=all_vocab)
predicted = all_Log_TFIDF_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

0.8266064790228359

In [33]:
print(metrics.classification_report(newsgroups_val_y, predicted,target_names=newsgroups_train.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.71      0.69      0.70       161
           comp.graphics       0.73      0.83      0.78       189
 comp.os.ms-windows.misc       0.77      0.73      0.75       197
comp.sys.ibm.pc.hardware       0.73      0.76      0.74       196
   comp.sys.mac.hardware       0.84      0.81      0.83       196
          comp.windows.x       0.86      0.77      0.81       202
            misc.forsale       0.78      0.91      0.84       206
               rec.autos       0.88      0.88      0.88       189
         rec.motorcycles       0.96      0.93      0.95       194
      rec.sport.baseball       0.90      0.95      0.93       192
        rec.sport.hockey       0.98      0.93      0.95       203
               sci.crypt       0.94      0.90      0.92       194
         sci.electronics       0.75      0.82      0.78       197
                 sci.med       0.88      0.81      0.85       196
         

In [34]:
all_SVM_Count_clf = BoG_model(newsgroups_train_X, newsgroups_train_y, vocab=all_vocab, clf = 'SVM')
predicted = all_SVM_Count_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

0.7565055762081785

In [35]:
print(metrics.classification_report(newsgroups_val_y, predicted,target_names=newsgroups_train.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.73      0.67      0.70       161
           comp.graphics       0.82      0.70      0.76       189
 comp.os.ms-windows.misc       0.70      0.62      0.66       197
comp.sys.ibm.pc.hardware       0.46      0.65      0.54       196
   comp.sys.mac.hardware       0.74      0.78      0.76       196
          comp.windows.x       0.81      0.71      0.76       202
            misc.forsale       0.80      0.79      0.79       206
               rec.autos       0.89      0.83      0.86       189
         rec.motorcycles       0.83      0.92      0.87       194
      rec.sport.baseball       0.90      0.87      0.89       192
        rec.sport.hockey       0.93      0.88      0.90       203
               sci.crypt       0.73      0.88      0.80       194
         sci.electronics       0.53      0.66      0.59       197
                 sci.med       0.85      0.74      0.79       196
         

In [36]:
all_SVM_TFIDF_clf = Tfidf_model(newsgroups_train_X, newsgroups_train_y, vocab=all_vocab, clf = 'SVM')
predicted = all_SVM_TFIDF_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

0.847052575677111

In [37]:
print(metrics.classification_report(newsgroups_val_y, predicted,target_names=newsgroups_train.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.76      0.76      0.76       161
           comp.graphics       0.77      0.84      0.81       189
 comp.os.ms-windows.misc       0.76      0.74      0.75       197
comp.sys.ibm.pc.hardware       0.77      0.76      0.76       196
   comp.sys.mac.hardware       0.84      0.85      0.85       196
          comp.windows.x       0.90      0.78      0.83       202
            misc.forsale       0.86      0.89      0.88       206
               rec.autos       0.89      0.88      0.88       189
         rec.motorcycles       0.94      0.94      0.94       194
      rec.sport.baseball       0.91      0.95      0.93       192
        rec.sport.hockey       0.98      0.99      0.98       203
               sci.crypt       0.92      0.91      0.91       194
         sci.electronics       0.80      0.78      0.79       197
                 sci.med       0.89      0.86      0.88       196
         

In [77]:
data_folder = "../data/raw/BBC News Summary/News Articles/"
entries = os.listdir(data_folder)
entries

['business', 'entertainment', 'politics', 'sport', 'tech']

In [85]:
bbc_df = pd.read_csv('../data/temp/all_data.csv')[['summary','type_code']]
X_train, X_test, y_train, y_test = train_test_split(bbc_df.summary, bbc_df.type_code, test_size=0.4, random_state=42)
X_val,X_test,y_val,y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [86]:
Log_Count_clf = BoG_model(X_train, y_train)
predicted = Log_Count_clf.predict(X_val)
np.mean(predicted == y_val)

C:\Users\Yang Li\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9595505617977528

In [87]:
Log_TFIDF_clf = Tfidf_model(X_train, y_train)
predicted = Log_TFIDF_clf.predict(X_val)
np.mean(predicted == y_val)

0.9685393258426966

In [90]:
SVM_Count_clf = BoG_model(X_train, y_train, clf = 'SVM')
predicted = SVM_Count_clf.predict(X_val)
np.mean(predicted == y_val)

0.9550561797752809

In [92]:
SVM_TFIDF_clf = Tfidf_model(X_train, y_train, clf = 'SVM')
predicted = SVM_TFIDF_clf.predict(X_val)
np.mean(predicted == y_val)

0.9820224719101124

In [106]:
len(SVM_TFIDF_clf['BagOfWord'].vocabulary_)

16243

In [93]:
with open('../data/temp/bbc_sum.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(bbc_df.summary.apply(lambda x: clean_text(x)+ '.')))

In [136]:
df_bbc = pd.read_csv('../data/temp/AutoPhrase_bbc.txt',error_bad_lines=False, delimiter= '\t',names = ['p','phrase'])
select_bbc = df_bbc.phrase[df_bbc.p >= 0.5]
select_bbc

0      manchester united
1           human rights
2        australian open
3      liberal democrats
4               mac mini
             ...        
936               damage
937          responsible
938              members
939                 fans
940                limit
Name: phrase, Length: 941, dtype: object

In [120]:
Log_Count_clf = BoG_model(X_train, y_train, vocab=select_bbc)
predicted = Log_Count_clf.predict(X_val)
np.mean(predicted == y_val)

C:\Users\Yang Li\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9438202247191011

In [130]:
Log_TFIDF_clf = Tfidf_model(X_train, y_train, vocab= select_bbc)
predicted = Log_TFIDF_clf.predict(X_val)
np.mean(predicted == y_val)

0.950561797752809

In [131]:
SVM_Count_clf = BoG_model(X_train, y_train,vocab= select_bbc, clf = 'SVM')
predicted = SVM_Count_clf.predict(X_val)
np.mean(predicted == y_val)

0.9415730337078652

In [132]:
SVM_TFIDF_clf = Tfidf_model(X_train, y_train,vocab= select_bbc, clf = 'SVM')
predicted = SVM_TFIDF_clf.predict(X_val)
np.mean(predicted == y_val)

0.9550561797752809

In [145]:
all_SVM_Count_clf = BoG_model(X_train, y_train,vocab= select_bbc, clf = 'SVM', combining = True)
predicted = all_SVM_Count_clf.predict(X_val)
np.mean(predicted == y_val)

0.9415730337078652

In [152]:
all_SVM_TFIDF_clf = Tfidf_model(X_train, y_train,vocab= select_bbc, clf = 'SVM', combining = True)
predicted = all_SVM_TFIDF_clf.predict(X_val)
np.mean(predicted == y_val)

0.9865168539325843

In [212]:
len(auto_ner_all_lst)

69357

In [175]:
df_raw = pd.read_csv('../data/temp/raw.csv')
NE_set = set()
for i in df_raw.NE:
    for j in i:
        NE_set.add(j)

In [183]:
import ast
df_raw.NE = df_raw.NE.apply(lambda x: ast.literal_eval(x))

In [189]:
ner_vocab = set(df_raw.NE.apply(lambda x: [re.sub(r"\:.+",'',i) for i in x]).explode().dropna())

In [198]:
auto_ner_all_lst = np.unique(list(ner_vocab) + list(select.phrase))

In [200]:
all_Log_Count_clf = BoG_model(newsgroups_train_X, newsgroups_train_y, vocab=auto_ner_all_lst)
predicted = all_Log_Count_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

C:\Users\Yang Li\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7679235262878386

In [201]:
all_Log_TFIDF_clf = Tfidf_model(newsgroups_train_X, newsgroups_train_y, vocab=auto_ner_all_lst)
predicted = all_Log_TFIDF_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

0.8048327137546468

In [203]:
all_SVM_Count_clf = BoG_model(newsgroups_train_X, newsgroups_train_y, vocab=auto_ner_all_lst, clf = 'SVM')
predicted = all_SVM_TFIDF_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

0.8138608603292619

In [205]:
all_SVM_TFIDF_clf = Tfidf_model(newsgroups_train_X, newsgroups_train_y, vocab=auto_ner_all_lst, clf = 'SVM')
predicted = all_SVM_TFIDF_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

0.8151885289431758

In [206]:
all_Log_Count_clf = BoG_model(newsgroups_train_X, newsgroups_train_y, vocab=auto_ner_all_lst, combining = True)
predicted = all_Log_Count_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

C:\Users\Yang Li\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7761550716941051

In [207]:
all_Log_TFIDF_clf = Tfidf_model(newsgroups_train_X, newsgroups_train_y, vocab=auto_ner_all_lst, combining = True)
predicted = all_Log_TFIDF_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

0.8266064790228359

In [208]:
all_SVM_Count_clf = BoG_model(newsgroups_train_X, newsgroups_train_y, vocab=auto_ner_all_lst, clf = 'SVM', combining = True)
predicted = all_SVM_TFIDF_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

0.8151885289431758

In [211]:
all_SVM_TFIDF_clf = Tfidf_model(newsgroups_train_X, newsgroups_train_y, vocab=auto_ner_all_lst, clf = 'SVM', combining = True)
predicted = all_SVM_TFIDF_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

0.8462559745087627